In [ ]:
import numpy as np

### Data

In [ ]:
import sklearn

In [ ]:
feature_encoder = sklearn.preprocessing.OrdinalEncoder()
feature_encoder.fit(X)

target_encoder = sklearn.preprocessing.LabelEncoder()
target_encoder.fit(y.squeeze())

X = feature_encoder.transform(X)
y = target_encoder.transform(y)

### Model

In [ ]:
from tensorflow import keras

In [ ]:
def build_model(input_shape, classes, n_layers = 5, n_nodes_l = 2, n_nodes_u = 10, optimizer = 'adam', learning_rate = 0.001):
    
    model = keras.Sequential()
    model.add(keras.Input(shape = (input_shape,)))
    for i in range(n_layers):
        model.add(keras.layers.Dense(np.random.choice([i for i in range(n_nodes_l, n_nodes_u)]), activation = np.random.choice(['relu', 'sigmoid'])))
    model.add(keras.layers.Dense(classes, activation = 'softmax'))

    if optimizer == 'sgd':
        optim = keras.optimizers.SGD(learning_rate = learning_rate)
    elif optimizer == 'adam':
        optim = keras.optimizers.Adam(learning_rate = learning_rate)
    
    model.compile(optimizer = optim, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    
    return model

In [ ]:
n_classes = len(np.unique(y_train))
input_shape = len(X_train[0])
n_layers = 5
n_nodes_l = 5
n_nodes_u = 25
optimizer = 'adam'
learning_rate = 0.01
epochs = 25
batch_size = 100

In [ ]:
model = build_model(
    input_shape,
    n_classes,
    n_layers = n_layers, 
    n_nodes_l = n_nodes_l,
    n_nodes_u = n_nodes_u, 
    optimizer = optimizer, 
    learning_rate = learning_rate
)

model.fit(X_train, y_train, epochs = epochs, batch_size = batch_size, validation_split = 0.2)
model.summary()

In [ ]:
y_pred = model(X_test)
y_pred = np.argmax(y_pred, axis = 1)

accuracy = sklearn.metrics.accuracy_score(y_pred, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

In [ ]:
### Benchmarking Against LIME, SHAP

In [ ]:
import os
import json
from explainer.explainer import Explainer
from explainer.metrics import markov_blanket, distribution_drift, average_distribution_drift, divergence_plot, fidelity_plot, mb_accuracy

In [ ]:
results = []
experiment_name = data_filename = os.path.join("./data/results", dataset_name)
ground_mb = markov_blanket(ground_bn, target_name)

In [ ]:
for _ in range(10):
    
    explainer = Explainer(
        model = model, 
        training_data = X_train, 
        feature_names = feature_names, 
        target_name = target_name, 
        n_samples = 5000, 
        rep_prob = 0.1
    )

    instance_idx = np.random.randint(X_test.shape[0])
    instance = X_test[instance_idx]
    
    explainer.log_data(instance)
    explainer.get_structures()

    explainer_mb_accuracy = mb_accuracy(ground_mb, explainer.bic_mb, target_name)

    # Features to freeze while randomizing the rest
    no_features = [] # randomize all features (baseline)
    explainer_features = list(explainer.bic_blanket) # freeze features returned from markov blanket of explainer
    
    no_new, no_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = no_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = 100
    )
    
    explainer_new, explainer_og = distribution_drift(
        instance = instance, 
        training_data = X_train, 
        feature_set = explainer_features, 
        feature_names = feature_names, 
        model = model, 
        n_trials = 100
    )
    
    no_avg, no_values = average_distribution_drift(no_og, no_new)
    explainer_avg, explainer_values = average_distribution_drift(explainer_og, explainer_new)
    
    js_values = np.stack((no_values, explainer_values))
    method_names = ['No Features', 'Explainer MB Features']
    
    print(f'No Features Divergence: {no_avg}')
    print(f'Explainer MB Features Divergence: {explainer_avg}')

    results.append({

        "dataset": dataset_name,

        "target": target_name,
        "features": feature_names,
        "mlp_accuracy": accuracy,
        
        "explainer_structure": list(explainer.bic_mb.edges()),
        
        "explainer_features": explainer_features,
        
        "no_new": no_new.tolist(),
        "explainer_new": explainer_new.tolist(),
        
        "no_og": no_og.tolist(),
        "explainer_og": explainer_og.tolist(),
        
        "no_values": no_values,
        "explainer_values": explainer_values,
        
        "no_avg": no_avg,
        "explainer_avg": explainer_avg,
        
    })

    with open(f'{experiment_name}.json', 'w') as f:

        json.dump(results, f, indent = 4)
    
    divergence_plot(js_values, method_names)
    
    fidelity_plot(no_new, no_og, method_names[0])
    fidelity_plot(explainer_new, explainer_og, method_names[1])